# J Software - Neural Networks
In Neural Networks in APL Manuel Alfonseca proposes a general tableau representation of a
Neural Network. In this representation all neurons feed into every other neuron. The problem
is his backpropagation routine does not seem to follow standard implementations. Brian Scott has a J formulation that tries to make full use of J syntax. 

## Backpropagation
Backpropagation is a method of weight adjustments on the connections between neurons. By
comparing calculated outputs to actual outputs a direct adjustment can be made on the last
hidden layer to the output layer neurons. Once that is accomplished an estimate of the error
propagated backward to earlier layers. 

## Neural Network Equations
### Neuron Activation Function
Various sigmoid type functions are used for two most popular are the standard sigmoid and the
hyperbolic. They differ in their range.
- $f(x) = tanh(x) = (e^{x} - e^{-x}) / (e^{x} + e^{-x})$ Range: [-1,1]
- $f(x) = 1 / (1 + e^{-x})$ Range: [0,1]

### Rao et al C++ Neural Networks and Fuzzy Logic implementation
1. input to hidden layer: $y_{j} = f( (\sum_{i=0}^{n} x_{i} M1_{i,j} ) + \theta_{j})$
2. hidden to output layer: $z_{j} = f( (\sum_{i=0}^{m} y_{i} M2_{i,j} ) + \tau_{j})$
3. output error component based on difference from calculated to actual: $e_{i} = z_{i} (1 - z_{i}) (actual_{i} - z_{i}) $
4. hidden layer error: $t_{i} = y_{i} (1 - y_{i}) ((\sum_{j=0}^{m} M2_{i,j} e_{j}) $
5. Adjustment matrix dM2: $dM2_{i,j} = \mu y_{i} e_{j}$
6. Adjustment matrix dM1: $dM1_{i,j} = \lambda x_{i} t_{j}$
7. Threshhold adjustments: $d \tau_{j} = \mu e_{j}$ and $d \theta-[J] = \lambda e_{j}$

Rao goes on to describe a momentum method of creating the adjustment matrices. Rao also separately accounts for the threshold value. 
### Notes
In the General Tableau method the threshold is an extra neuron that has a weight to every other neuron and the threshold value is therefore adjustable to each neuron by adjusting the weight from the threshold neuron.

In equations 1 or 2 for J purposes this is just the matrix multiplication of a matrix and a vector. 

In equations 3 and 4 the z (1-z) and y (1-y) are the derivative formylations of the sigmoid function. Brian Schott's implementation provides separate function definition for these derivatives.

Equation 4 represents an outer product (x */ y in J) 

The Rao formulation does not allow feed back. The layers are unidirectional: inputs feed hidden which feed outputs.

Does the dual matrix implementation as shown in Rao's formulation have an equivalent single operation implementation in the General Tableau.

## Rao implementation in J
### activation functions in J

In [32]:
NB. sigmoid function
sigmoid   =: %@:>:@^@-   NB. Brian Schott implementation
esigmoid  =: 13 : '% 1 + (^ (-y))'  NB. more mathematical looking (ie. less J looking)

NB. the thing about J is you have to be careful about combining operators because the 
NB. system will treat them as 'forks' which is a J shorthand of combining operators into 
NB. a hierarchy that will operator in specific ways on the left and righthand operands.

NB. tanh
tanh =: 7&o.

NB. derivatives of the above functions
derivSig =: ( * -.)&sigmoid   NB. Brian Schott implementation sigmoid * (1 - sigmoid)

derivTanh =: -.@*:@tanh       NB. Brian Schott implementation 1 - tanh^2

### Layer Weight Matrices
Define M1 and M2 to be randomized weights. Brian Schott's implementation has a set of easy J functions to accomplish this

In [2]:
NB. generating random initial connection weights************
shapeNodes =: 2&(<@:+&1 0\)
initNodes  =:  shapeNodes@] $ each [

times10    =: 10&*
inflate    =: >:@+: NB. @times10
NB. rn takes values between 0.1(0.1)2
NB. and produces random fractions between -y. and +y.
NB. the fractions are produced with 1 decimal accuracy
NB. I think
rn         =: (?--:@<:)@ inflate &.: times10
NB. rn 0.4

NB. rn_nodes produces random starting weights for
NB. a feed-forward neural network
NB. the optional left argument controls the range of
NB. the random starting weights
rn_nodes   =: 1&rn_nodes : ([: (rn each) initNodes)


In [3]:
NB. Now it's easy to create the matrices with randomized weights. In this version they are
NB. In boxed form the input will be a vector of #input_neurons, #hidden, #output
rn_nodes 10 5 2

┌────────────────────────┬─────────┐
│ 0.3  0.3  0.1  0.3 _0.8│_0.4    1│
│   1  0.5    0  0.8 _0.5│_0.5 _0.9│
│ 0.5  0.3  0.8 _0.9  0.5│  _1  0.3│
│_0.2   _1  0.4  0.2  0.9│ 0.5  0.5│
│  _1  0.4  0.3  0.8    1│_0.8    0│
│_0.8 _0.8 _0.3  0.4  0.9│_0.5  0.8│
│   0  0.8  0.4  0.6 _0.3│         │
│_0.3    0 _0.4  0.9  0.8│         │
│ 0.9  0.2  0.4 _0.4    1│         │
│_0.8 _0.1 _0.6  0.9    1│         │
│ 0.5 _0.6  0.5    1    1│         │
└────────────────────────┴─────────┘


In [4]:
NB. to assign them into matrix variables
'M1 M2' =: rn_nodes 10 5 2

In [5]:
NB. M1 =
M1

 0.7 _0.6  0.3 _0.3    1
_0.4  0.1 _0.3  0.8 _0.1
 0.2  0.9 _0.1 _0.8  0.4
   0  0.4 _0.5    0  0.9
_0.1  0.9  0.1 _0.5  0.6
 0.3  0.3  0.5  0.3  0.3
_0.9 _0.7 _0.9 _0.6  0.6
 0.2  0.4  0.4 _0.2  0.5
_0.9 _0.8 _0.3  0.5  0.6
_0.6 _0.4 _0.4  0.6  0.8
  _1  0.5  0.2 _0.2    0


In [6]:
NB. M2 =
M2

 0.9 _0.5
  _1  0.7
_0.4  0.1
_0.1    1
_0.1  0.8
 0.1  0.1


### implementation note
Rather than separately account for the threshold it makes sense to include it as an extra node to each layer. That way the thresholds can be adjust through backpropagation as well. In the separate weight matrix representation that means adding a node to the input layer and hidden layer and then appending a 1 to the input vector and appending a 1 to the output vector of the hidden nodes. I will just add it to the front of each vector as this is what is done in the Tableau method.

So now we need to redefine the layers to add 1 to the inputs and 1 to the hiddens, then regenerate the initial random weights. For the above example where 10 input nodes, 5 hidden nodes and 2 output nodes were created (10 5 2) it will now become 11 6 2. To keep an account of actual inputs I will create a J vector by explicitly adding 1 in the definition:

In [22]:
X =: 1+i.10
]Y =: sigmoid (1,X) +/ . * M1

1.69628e_11 0.768525 0.00449627 0.689974 1


In [10]:
]Z =: sigmoid (1,Y) +/ . * M2

0.650933 0.558069


In [12]:
P =: 0.75 0.65
]eh =: Z * (1 - Z) * (P - Z)

0.0225099 0.0226728


In [27]:
]ti =: (1,Y) *(1 - 1,Y) * M2 +/ . * eh

0 _1.12615e_13 _0.00117051 9.14132e_5 0.00342857 5.94969e_15


In [20]:
lambda =: 0.2
mu =: 0.2
]dM2 =: mu * (1,Y) */ eh

 0.00450198  0.00453456
7.63661e_14 7.69187e_14
 0.00345989  0.00348492
 2.02421e_5  2.03886e_5
 0.00310625  0.00312873
 0.00450198  0.00453456


In [28]:
NB. drop the first column because we don't propagate back the threshold neuron
]dM1 =: lambda * (1,X) */ }. ti    

 _2.2523e_14 _0.000234102  1.82826e_5 0.000685713 1.18994e_15
 _2.2523e_14 _0.000234102  1.82826e_5 0.000685713 1.18994e_15
_4.50461e_14 _0.000468203  3.65653e_5  0.00137143 2.37988e_15
_6.75691e_14 _0.000702305  5.48479e_5  0.00205714 3.56981e_15
_9.00922e_14 _0.000936407  7.31306e_5  0.00274285 4.75975e_15
_1.12615e_13  _0.00117051  9.14132e_5  0.00342857 5.94969e_15
_1.35138e_13  _0.00140461 0.000109696  0.00411428 7.13963e_15
_1.57661e_13  _0.00163871 0.000127979  0.00479999 8.32956e_15
_1.80184e_13  _0.00187281 0.000146261  0.00548571  9.5195e_15
_2.02707e_13  _0.00210692 0.000164544  0.00617142 1.07094e_14
 _2.2523e_13  _0.00234102 0.000182826  0.00685713 1.18994e_14


In [29]:
M2 =: M2 + dM2
M1 =: M1 + dM1
]Y =: sigmoid (1,X) +/ . * M1

1.69628e_11 0.752061 0.00452797 0.743585 1


In [30]:
]Z =: sigmoid (1,Y) +/ . * M2

0.657505 0.575087


# Notes
It appears from this small treatment that the thresholds even though they are combined into the weight matrix still are treated somewhat separately. You must account for the nodes on the inputs to each layer and you must drop their column because they do not propagate to the next layer.

The current adjustment does not move the calculated value very quickly toward the desired actual value. This brings up the question of what are the best lambda and mu values to use?Granted the above example is contrived and there is only one output. The idea is that each separate input/output combination moves the system ever so slightly toward the goal of trying to be close for every input/output combination. This is driven over multiple combinations and multiple epochs in the hope of arriving at a semi-optimal solution.

In treating the layer matrices separately there is no feedback allowed. The system will calculate an answer in all cases. In the Tableau approach every neuron is connected to every other neuron and therefore feedback arises and a steady state must be reached before an answer is obtained. 

In the Tableau method, how is backpropagation obtained since feedback will come into play. Does the backpropagation require a steady state to be reached as well?

# Tolerance
The following function was found on the J mailing list at: 

http://www.jsoftware.com/pipermail/programming/2013-May/032730.html

It will zero values that are close to zero.

In [31]:
clean=: (* |@*)&.+.
dM1
clean dM1

 _2.2523e_14 _0.000234102  1.82826e_5 0.000685713 1.18994e_15
 _2.2523e_14 _0.000234102  1.82826e_5 0.000685713 1.18994e_15
_4.50461e_14 _0.000468203  3.65653e_5  0.00137143 2.37988e_15
_6.75691e_14 _0.000702305  5.48479e_5  0.00205714 3.56981e_15
_9.00922e_14 _0.000936407  7.31306e_5  0.00274285 4.75975e_15
_1.12615e_13  _0.00117051  9.14132e_5  0.00342857 5.94969e_15
_1.35138e_13  _0.00140461 0.000109696  0.00411428 7.13963e_15
_1.57661e_13  _0.00163871 0.000127979  0.00479999 8.32956e_15
_1.80184e_13  _0.00187281 0.000146261  0.00548571  9.5195e_15
_2.02707e_13  _0.00210692 0.000164544  0.00617142 1.07094e_14
 _2.2523e_13  _0.00234102 0.000182826  0.00685713 1.18994e_14
           0 _0.000234102  1.82826e_5 0.000685713 0
           0 _0.000234102  1.82826e_5 0.000685713 0
           0 _0.000468203  3.65653e_5  0.00137143 0
_6.75691e_14 _0.000702305  5.48479e_5  0.00205714 0
_9.00922e_14 _0.000936407  7.31306e_5  0.00274285 0
_1.12615e_13  _0.00117051  9.14132e_5  0.00342857 0
_1.351

# References
1. Rao, Valluru, and Hayagriva V. Rao. *C Neural Networks and Fuzzy Logic.* New York: MIS:Press, 1993. Print.
2. Schott, Brian. "User:Brian Schott/code/feedforwardNN." - J Wiki. J Software Inc., 8 Dec. 2008. Web. 23 Feb. 2016. http://code.jsoftware.com/wiki/User:Brian_Schott/code/feedforwardNN.
3. Alfonseca, Manuel. "Neural Networks in APL." SIGAPL APL Quote Quad ACM SIGAPL APL Quote Quad 20.4 (1990): 2-6. Web
4. Lane, Kenneth M., and Richard D. Neidinger. "Neural Networks from Idea to Implementation." SIGAPL APL Quote Quad ACM SIGAPL APL Quote Quad 25.3 (1995): 27-37. Web
5. Hui, Roger. "[Jprogramming] Clean." J Programming Forum. J Software Inc., 27 May 2014. Web. 23 Feb. 2016. http://www.jsoftware.com/pipermail/programming/2013-May/032730.html.